# Issue Commands to Change Settings

https://github.com/stilldavid/gopro-utils

In [1]:
import bs4

import context

from gopro_helper import gopro
from gopro_helper import api
from gopro_helper import commands

# Camera Current Settings

In [27]:
k = gopro.GoProStatus(interval=1)

In [65]:
k.stop()

# Basic Commands

In [2]:
commands.set_mode_photo()

In [38]:
commands.set_mode_video()

In [53]:
commands.shutter_capture()

In [52]:
commands.shutter_stop()

In [62]:
%%time

urls = commands.get_data_urls()

CPU times: user 1.52 s, sys: 32 ms, total: 1.56 s
Wall time: 8.92 s


In [63]:
len(urls)

8454

---

In [13]:
commands.delete_file(files[6]['url'])

'http://10.5.5.9/gp/gpControl/command/storage/delete?p=/100GOPRO/G0010007.JPG'